<a href="https://colab.research.google.com/github/nrarifahmed/nvidia1/blob/main/cuMLNvidia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
 !nvidia-smi

Mon May 24 12:10:05 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

https://colab.research.google.com/drive/1ddEnBj4jh0G05UR0FuAtWLfnCQ97nwWP?usp=sharing#scrollTo=2k_Av5fEDkbS

https://rapids.ai/index.html

In [5]:
# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

import sys, os, shutil

sys.path.append('/usr/local/lib/python3.7/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ["CONDA_PREFIX"] = "/usr/local"
for so in ['cudf', 'rmm', 'nccl', 'cuml', 'cugraph', 'xgboost', 'cuspatial']:
  fn = 'lib'+so+'.so'
  source_fn = '/usr/local/lib/'+fn
  dest_fn = '/usr/lib/'+fn
  if os.path.exists(source_fn):
    print(f'Copying {source_fn} to {dest_fn}')
    shutil.copyfile(source_fn, dest_fn)
# fix for BlazingSQL import issue
# ImportError: /usr/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.26' not found (required by /usr/local/lib/python3.7/site-packages/../../libblazingsql-engine.so)
if not os.path.exists('/usr/lib64'):
    os.makedirs('/usr/lib64')
for so_file in os.listdir('/usr/local/lib'):
  if 'libstdc' in so_file:
    shutil.copyfile('/usr/local/lib/'+so_file, '/usr/lib64/'+so_file)
    shutil.copyfile('/usr/local/lib/'+so_file, '/usr/lib/x86_64-linux-gnu/'+so_file)

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 238, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 238 (delta 36), reused 11 (delta 5), pack-reused 171
Receiving objects: 100% (238/238), 73.89 KiB | 18.47 MiB/s, done.
Resolving deltas: 100% (98/98), done.
PLEASE READ
********************************************************************************************************
Changes:
1. IMPORTANT SCRIPT CHANGES: Colab has updated to Python 3.7, and now runs our STABLE and NIGHTLY versions (0.19 and 0.20)!  PLEASE update your older install script code as follows:
	!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.19

	import sys, os, shutil

	sys.path.append('/usr/local/lib/python3.7/site-packages/')
	os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
	os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
	os.environ['CONDA_PREFIX'] = '/usr/local'
	for so in ['cudf', 'rmm',

In [6]:
import cudf
from cuml import make_regression, train_test_split
from cuml.linear_model import LinearRegression as cuLinearRegression
from cuml.metrics.regression import r2_score
from sklearn.linear_model import LinearRegression as skLinearRegression

In [7]:
n_samples = 2 ** 10
n_features = 399
random_state = 23

In [8]:
# Lets' generate some random regression data
%%time

X,y = make_regression(n_samples = n_samples,
                      n_features = n_features,
                      random_state =  random_state)

X = cudf.DataFrame(X)
y = cudf.DataFrame(y)[0]

X_cudf,X_cudf_test,y_cudf,y_cudf_test = train_test_split(X,y,test_size=0.2,random_state=random_state)

CPU times: user 2.86 s, sys: 1.23 s, total: 4.09 s
Wall time: 12.6 s


In [9]:
# Copy dataset from GPU to host memory
# This is done to later compare GPU & CPU results

X_train = X_cudf.to_pandas()
X_test = X_cudf_test.to_pandas()
y_train = y_cudf.to_pandas()
y_test = y_cudf_test.to_pandas()

In [10]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398
845,2.055562,-0.933736,0.497016,0.620149,-0.476680,-2.029445,0.123182,-0.641415,-0.553967,2.717702,0.860558,-0.313322,-1.051106,-1.380937,0.417878,-1.344837,-1.589228,2.148409,-1.095631,-0.140356,-0.079392,-0.659669,-0.360686,1.248658,-0.131434,-0.862661,0.005914,-0.601110,0.131767,0.742445,1.836354,-0.371129,-2.653161,-0.857406,-0.838788,-0.122874,0.531857,-0.696591,-0.073658,-0.223529,...,-1.144055,0.661332,0.625272,0.071821,-0.711666,0.647322,0.342235,-0.902038,-2.142352,1.303072,-1.025851,-0.403434,-2.296029,-1.235699,-0.070463,-0.645626,0.443654,0.981243,1.360011,-1.134194,0.520725,-0.123204,0.187861,-0.158220,0.078023,-0.184462,-1.095767,0.206128,-0.535497,-0.754671,-1.550962,2.538601,0.166509,1.863421,-0.352203,0.469129,-0.834900,-0.453884,1.550886,0.570708
480,-0.448590,1.736363,-0.136029,-0.022542,0.416618,-0.013258,-0.254864,0.336946,0.080474,-1.090549,-0.927134,1.667028,0.575787,-0.912464,0.348150,-0.145277,0.361353,0.701353,0.788017,-0.160181,1.499050,-2.972301,-0.755454,-1.087193,0.805894,-0.899627,-0.705622,1.018941,-0.489621,-0.082309,-0.371689,0.865737,-0.227379,1.083263,-0.304605,-0.760506,0.438162,-0.371897,1.617065,-0.390182,...,-1.885694,-1.417606,1.484554,1.966353,-0.387893,-1.300339,-0.103005,-0.046219,0.427647,-0.063938,-0.503205,1.253227,-1.085402,-0.131801,-2.345319,0.593710,1.220944,-0.603804,-0.205882,-1.609632,1.400760,-0.960835,0.122997,-1.220349,-0.838515,0.907545,0.869625,1.744500,-1.839492,-0.400698,-0.666103,0.192153,-0.032492,1.232339,0.966777,-0.450480,0.921368,-1.218761,0.573733,-0.126662
162,0.569151,-0.333358,-0.712302,0.676326,0.471869,0.722293,0.726290,-0.042724,0.303953,-0.012106,0.172228,0.166787,-2.062305,-1.726970,0.019084,-0.455605,-1.120517,0.702563,0.967574,-0.728839,-0.130154,1.152042,-0.912612,-0.569285,0.872799,-0.823302,-0.773043,-0.209472,0.795978,0.570768,-0.351556,-0.162365,-1.589545,0.698792,-0.082682,0.844782,-0.215189,0.202043,0.162839,-0.812618,...,1.263705,1.174916,0.619487,0.648082,-0.132168,-0.568357,1.199464,-1.199975,0.510640,-0.080806,0.733008,1.259133,-0.481431,-0.545911,1.100202,-0.375840,0.116208,0.693646,-0.953451,-1.012596,-1.226177,0.923681,0.338422,1.876497,-0.435721,1.367830,1.020849,1.914616,0.419945,1.050397,0.067205,0.939033,-0.855822,-0.434755,0.326109,-0.423419,-1.240627,0.734930,-0.179995,1.380971
49,0.236424,-0.600488,-1.037158,-0.069217,-0.675002,-0.724204,-1.039712,0.907647,1.059449,-0.061243,0.814336,0.304064,-0.159710,-0.810689,0.760082,-0.547184,-1.364508,0.139291,1.189483,0.199203,-1.382915,1.810462,-0.432328,-0.546016,0.611132,0.934809,0.545220,-0.206231,-1.417910,-1.023365,0.001771,0.141826,-0.490810,0.500067,0.164120,-0.129105,0.817416,-0.832125,-0.968688,-0.247358,...,0.144684,1.683117,-1.290303,2.003380,-0.926158,0.916690,0.528824,1.821189,0.634556,0.591865,0.035116,1.125213,0.415175,-0.733702,-0.775972,1.209593,0.380568,-0.351402,-2.038898,1.416754,1.702480,-0.588385,0.634842,1.032439,0.145111,-0.210908,-0.119754,-0.565198,0.312457,0.727650,1.027451,-0.505768,0.044367,0.559193,0.462736,1.526734,-0.822133,0.555926,-0.368712,-0.093048
201,0.536586,0.935002,0.705363,1.014163,0.033458,-1.371604,0.980535,-0.024624,0.509206,-0.162935,-0.298795,2.148899,0.750777,-1.548709,1.032921,-0.440200,-0.955357,0.570711,0.700902,1.090451,-0.012428,0.713988,0.831759,1.034216,-0.920832,3.683870,1.572410,0.345078,0.684440,-0.138149,-0.576254,-0.676192,0.837158,-1.499398,-0.369281,2.387580,-0.033785,-0.454792,-2.076226,-0.956245,...,0.771048,-1.513548,-0.130502,1.276655,-0.686441,0.270156,-0.614225,0.565890,0.483493,0.345290,0.055891,0.002791,3.333124,0.712124,-2.344680,0.027007,-1.321990,-0.292132,1.613297,-0.685362,-0.266351,-0.859685,0.874158,-1.133795,-0.681455,0.670882,-0.321680,-1.71

In [14]:
X_train.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398
count,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,...,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000,820.000000
mean,-0.023042,0.005178,-0.006164,0.015003,0.033919,-0.035150,-0.022223,0.042814,-0.026675,0.019944,0.015572,0.030549,-0.005321,-0.025461,-0.006296,0.049247,-0.043613,0.008505,-0.044951,-0.020120,0.022786,0.023015,-0.038300,-0.009633,0.022322,0.044519,0.024940,0.059132,0.027045,0.031805,-0.004045,-0.062949,0.043250,-0.051670,-0.021753,-0.002472,-0.000059,0.026031,-0.026002,-0.025037,...,-0.028761,0.029603,0.009803,0.006346,-0.027600,-0.037388,-0.059703,0.006228,-0.021596,-0.015394,0.020802,-0.067336,-0.016517,0.071860,0.027797,-0.046105,0.051559,0.007166,-0.061347,0.018629,0.015784,0.011992,0.023666,-0.116180,0.021084,0.028121,-0.019863,0.016505,0.050204,-0.021255,0.042421,-0.017465,-0.064813,-0.026468,0.083056,-0.041061,-0.029321,-0.059744,0.056459,-0.012753
std,0.963347,0.978566,0.994408,1.018789,0.992241,0.989216,1.004316,0.994737,0.973753,1.015698,0.991712,0.991659,0.980523,1.010575,1.002383,0.980225,0.999383,0.986869,1.031586,0.950619,0.983035,0.999877,1.035386,1.000052,1.024520,0.972535,1.013830,0.968843,1.020449,1.015795,0.981226,1.019611,1.036808,0.999371,1.012150,1.011982,1.016574,1.039826,0.984201,0.993550,...,0.992386,1.030695,0.955814,0.983729,0.993385,0.992541,0.986072,1.001151,1.014653,1.006241,0.978383,0.992375,0.967008,1.001619,0.987736,1.030573,0.998830,0.983449,1.016970,0.983192,1.065491,0.987264,1.005208,1.039847,0.964888,0.983499,0.984606,0.997948,0.974278,1.013827,0.985881,1.002822,0.972544,1.002425,0.978407,0.962110,0.998465,1.022061,1.002311,0.996692
min,-2.635024,-3.288889,-3.827111,-2.821422,-3.315894,-2.999923,-3.033219,-2.835010,-3.070719,-2.949272,-2.629420,-3.019743,-2.864982,-3.097392,-2.618501,-3.139254,-3.562185,-3.084054,-3.004707,-3.143399,-3.221824,-3.703752,-3.659472,-3.522635,-2.955364,-2.870863,-2.875204,-2.920392,-3.312007,-2.874605,-3.718862,-3.163097,-3.354109,-4.467167,-3.555304,-3.091891,-3.193046,-3.388127,-3.266604,-3.637738,...,-2.904818,-3.203330,-3.048476,-3.285143,-2.998003,-2.930471,-2.891410,-2.792073,-3.366362,-3.050051,-2.966328,-4.231245,-2.920286,-4.088656,-3.138756,-3.731404,-3.061174,-3.801624,-4.062862,-3.919347,-3.245381,-3.542094,-3.019219,-3.429781,-2.712014,-2.982888,-2.721092,-3.942121,-3.179962,-2.865566,-3.672620,-3.083125,-3.272066,-3.173814,-2.426709,-2.721319,-3.267492,-3.078031,-3.426399,-3.111436
25%,-0.675190,-0.632819,-0.699350,-0.714756,-0.623041,-0.643367,-0.713665,-0.649042,-0.713695,-0.665637,-0.684107,-0.704655,-0.720088,-0.680070,-0.678941,-0.593123,-0.702063,-0.677166,-0.719577,-0.623550,-0.659905,-0.664719,-0.756878,-0.708958,-0.662000,-0.635357,-0.670025,-0.592266,-0.680626,-0.642829,-0.666213,-0.729053,-0.633316,-0.727437,-0.665169,-0.706336,-0.699503,-0.714867,-0.712577,-0.750812,...,-0.685298,-0.628087,-0.639406,-0.665475,-0.693145,-0.726213,-0.719602,-0.669024,-0.708991,-0.770192,-0.631474,-0.728080,-

# cuML Model
### Fit, predict and evaluate 

In [17]:
# Fit 
%%time

ols_cuml = cuLinearRegression(fit_intercept=True,
                              normalize = True,
                              algorithm = 'eig')

ols_cuml.fit(X_cudf,y_cudf)

CPU times: user 55.4 ms, sys: 507 µs, total: 55.9 ms
Wall time: 89.8 ms


In [18]:
# Transform
%%time

predict_cuml = ols_cuml.predict(X_cudf_test)

CPU times: user 29.1 ms, sys: 599 µs, total: 29.7 ms
Wall time: 30.2 ms


In [19]:
# Evaluate
%%time
r2_score_cuml = r2_score(y_cudf_test, predict_cuml)

CPU times: user 1.33 ms, sys: 839 µs, total: 2.17 ms
Wall time: 7.99 ms


## sklearn Model

## Fit, predict and evaluate

In [20]:
# Fit 
%%time

ols_sk = skLinearRegression(fit_intercept=True,
                              normalize = True,
                             n_jobs = 1)

ols_sk.fit(X_train, y_train)

CPU times: user 63.8 ms, sys: 34.5 ms, total: 98.2 ms
Wall time: 80.6 ms


In [21]:
# Transform
%%time
predict_sk = ols_sk.predict(X_test)

CPU times: user 2.21 ms, sys: 783 µs, total: 2.99 ms
Wall time: 8.2 ms


In [22]:
# Evaluate
%%time
r2_score_sk = r2_score(y_cudf_test, predict_sk)

CPU times: user 1.73 ms, sys: 2.01 ms, total: 3.74 ms
Wall time: 5.15 ms


In [23]:
print("R^2 score (SKL):  %s" % r2_score_sk)
print("R^2 score (cuML): %s" % r2_score_cuml)

R^2 score (SKL):  1.0
R^2 score (cuML): 1.0
